In [1]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import date
import schedule

In [2]:
def updateMenu():
    reqheaders = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    todayMenu = requests.request('GET','https://front.cjfreshmeal.co.kr/meal/v1/today-all-meal?storeIdx=6442')
    todayMenuLunch = json.loads(todayMenu.text)['data']['2']
    menu = [0] * 5
    for i in range(5):
        menu[i] = {
            'name':todayMenuLunch[i]['name'],
            'side':todayMenuLunch[i]['side'],
            'corner':todayMenuLunch[i]['corner'],
            'kcal':todayMenuLunch[i]['kcal'],
            'thumbnailurl':todayMenuLunch[i]['thumbnailUrl']
        }
    return menu

In [3]:
def getRenderString(menu):
    renderstring = ''
    renderstring += '# ' + date.today().strftime("%Y-%m-%d") + ' 오늘의 점심\n'
    for i in range(5):
        renderstring += '## ' + str(i+1) + '. ' + menu[i]['corner'] + '\n'
        renderstring += '### ' + '**' + menu[i]['name'] + '**\n'
        renderstring += menu[i]['side'] + '\n'
        renderstring += str(int(menu[i]['kcal'])) + 'kcal\n'
        renderstring += '![' + menu[i]['name'] + '](' + menu[i]['thumbnailurl'] + ')\n'
        renderstring += '---\n'
    return renderstring

In [4]:
def post(menu, url_hook):
    headers = {'Content-Type':'application/json'}
    data = {'text':'bot'}
    response = requests.post(url_hook, headers=headers, data=json.dumps({"text":menu}))

In [5]:
def dailyUpdateAndPost(url_hook):
    post(getRenderString(updateMenu()), url_hook)

In [9]:
def dailyMenuUpdate():
    url_hook_file = './url_hook.txt'
    with open(url_hook_file, 'rt') as f:
        url_hook = f.read()
    dailyUpdateAndPost(url_hook)

In [10]:
if __name__ == '__main__':
    schedule.every().day.at("10:30").do(dailyMenuUpdate)

In [11]:
dailyMenuUpdate()